# NLP based query of Airbnb properties with Asymmetric Semantic Search
- https://www.seanxwang.com/2022/11/nlp-based-query-of-airbnb-properties.html

In [1]:
import pandas as pd
import pickle
import warnings
warnings.filterwarnings(action='once')
pd.set_option('display.max_colwidth', None)

## Get Sample Listing Data and Reviews
- use LA area listing for prototype
- simulate the user selected filtering to choose from a subset of listings
- use more recent reviews

In [2]:
# get sample listing data
listing_url='http://data.insideairbnb.com/united-states/ca/los-angeles/2022-09-09/data/listings.csv.gz'
listings = pd.read_csv(listing_url, compression='gzip', index_col=0)
print("size of sample listing data", listings.shape)
print(listings.columns)

size of sample listing data (45815, 74)
Index(['listing_url', 'scrape_id', 'last_scraped', 'source', 'name',
       'description', 'neighborhood_overview', 'picture_url', 'host_id',
       'host_url', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_thumbnail_url', 'host_picture_url',
       'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'neighbourhood',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'latitude',
       'longitude', 'property_type', 'room_type', 'accommodates', 'bathrooms',
       'bathrooms_text', 'bedrooms', 'beds', 'amenities', 'price',
       'minimum_nights', 'maximum_nights', 'minimum_minimum_nights',
       'maximum_minimum_nights', 'minimum_maximum_nights',
       'maximum_maximum_nights', 'minimum_nights_avg_ntm',
  

In [3]:
# Simulate user filter and selected listings, change index name to match with reviews (for later merge)
listings_filtered = listings[   (listings['amenities'].str.contains('Wifi')) & 
                                (listings['has_availability']=='t') &
                                (listings['review_scores_rating']>4.88)
                            ]

# change index name to match with reviews (for later merge)
listings_filtered = listings_filtered.rename_axis('listing_id')
print('subset of listings selected:', listings_filtered.shape)
listings_filtered.head()

subset of listings selected: (14846, 74)


,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
listing_id,,,,,,,,,,,,,,,,,,,,,
183319,https://www.airbnb.com/rooms/183319,20220909140055,2022-09-10,previous scrape,Panoramic Ocean View Venice Beach,"Craftsmen style penthouse with ocean view from all rooms. Park ave location is center of idyllic Venice walk street neighborhood.<br /><br /><b>The space</b><br />Large top floor [20'x60'] in Craftsmen Style Bungalow on most beautiful,original, and sought after walk street. Central great room with open kitchen flanked by two huge, studio style bedrooms. Steps to the beach. <br /><br />Extremely quiet with new double pane windows. Perfect situation for people wanting to work at home. Not the usual cluttered sublet, but furnished with everything for comfort. <br /><br />Convenient for shopping and entertainment with Abbott Kinney, Main Street, and Santa Monica in walking distance. Arguably one of the best, few, quiet ocean view apartments in Venice.<br /><br />Internet and utilities included.<br /><br />60 nights minimum <br /><br />Necessary inquiries please!<br /><br /><b>Guest access</b><br />This apartment is the entire top floor of a 3 story craftsmen type building.",About as close to the beach as you can live in Venice with out being on the boardwalk.,https://a0.muscache.com/pictures/1170205/e259632f_original.jpg,867995,https://www.airbnb.com/users/show/867995,...,5.00,5.00,5.00,NaN,f,2,2,0,0,0.02
51546,https://www.airbnb.com/rooms/51546,20220909140055,2022-09-10,city scrape,Cool Pad Under the Hollywood Sign,"*** Please note, after January 2, 2020 we can only accept bookings of 31 days or more, but at the discounted rate of $100 per night. Sorry to say, the laws in Los Angeles have changed and so I can only book by the month.***<br /><br />This is an Entire Apartment in Beachwood Village. Good choice for families, working professionals, or traveling friends. Enjoy easy access, relaxing atmosphere, off-street parking, many amenities, and a great central location tucked away in the Hollywood Hills.<br /><br /><b>The space</b><br />Welcome to our spacious 1 Bedroom Guest Apartment, in a restored 1920's Spanish Duplex. This is a cool place on a tiny little street under The Hollywood Sign. The area is known as Beachwood Canyon; We are located in the Village, near Beachwood Market. Downtown Hollywood is just minutes away but here, you are totally removed from traffic, hustle and bustle. This is an ideal location for visitors and business travelers.<br /><br />It has a living room, formal dining r","Beachwood Canyon is peaceful, natural, and lovely. It's hard to imagine you are in the heart of Los Angeles. We enjoy our village with its market, cafe, and friendly neighbors. The picture of the Hollywood Sign you see most often, could have been taken at my corner.",https://a0.muscache.com/pictures/3871329/c992db48_original.jpg,237114,https://www.airbnb.com/users/show/237114,...,4.99,4.96,4.89,NaN,f,1,1,0,0,1.30
185557,https://www.airbnb.com/rooms/185557,20220909140055,2022-09-09,city scrape,"Private Room in Home to FEMALE, pls (Purple Room)","<b>The space</b><br />Hello, Ladies!~ I am a female homeowner in L.A. and I have 3 private rooms, for one Female guest each.( NO Exceptions..In Compliance with AirBnB as well as CA. housing laws!) Both the ""Red Room"" and the ""Purple Room"" feature: Double bed in Purple Room, and high bed in Red Room, closet, antique dresser, desk and chair, desk lamp, unique decorations, three windows each, carpet and Small fridge in purple room, hardwood floor and good size mini fridge in red room, and full length mirror in each room. NO Storage space for your Stuff..so pl

In [4]:
# get review data
url = 'http://data.insideairbnb.com/united-states/ca/los-angeles/2022-09-09/data/reviews.csv.gz'
reviews = pd.read_csv(url,compression='gzip',index_col=0)
print("size of all review data", reviews.shape)
reviews.head()

size of all review data (1612656, 5)


,id,date,reviewer_id,reviewer_name,comments
listing_id,,,,,
109,449036,2011-08-15,927861,Edwin,The host canceled my reservation the day before arrival.
109,74506539,2016-05-15,22509885,Jenn,"Me and two friends stayed for four and a half months. It was a great place to stay! The apartment was very comfortable and I really enjoyed having the park with running path across the street. The only downside was it wasn't within walking distance to restaurants, bars, or coffee shops. But they are a short drive away. Overall, great stay!"
2708,13994902,2014-06-09,10905424,Kuberan,i had a wonderful stay. Everything from start to the end was perfect.<br/><br/>Will come back again.
183319,988343,2012-03-11,1219947,Gabriella,"very nice and cozy apartment, well located in Venice Beach. I had a great experience!"
183319,184002210,2017-08-18,70046039,Yan,We loved the experience. The location couldn't have been better. The accommodation is spacious and comfortable. Will definitely come back again!


In [5]:
# filter review by date
reviews_recent = reviews[reviews['date']>'2022-06-01']

# merge review data with listing to select subset of reviews corresponding to select listings
reviews_selected = reviews_recent.merge(listings_filtered, how='inner', left_index=True, right_index=True)
print("size of selected review data", reviews_selected.shape)

size of selected review data (57863, 79)


In [6]:
# listings have variable number of reviews
reviews_selected.index.value_counts()

22215734              57
39471761              50
16827829              47
16275577              47
25357098              44
                      ..
668155883570826348     1
619019807671863089     1
39524055               1
472748                 1
54209559               1
Name: listing_id, Length: 9416, dtype: int64

## Search Database - Generate reviews embedding with transformer model

In [7]:
# load transformer model for Asymmetric Semantic Search
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('msmarco-distilbert-base-v4')

In [8]:
# generate embeddings for review 
reviews_selected_list = reviews_selected['comments'].fillna('').values.tolist()
review_embedding = model.encode(reviews_selected_list)
print("reviews embedding size:", review_embedding.shape)

reviews embedding size: (57863, 768)


In [9]:
# save embedding
with open("../data/review_embedding_msmarco_v4.pkl", "wb") as reviews_f:
    pickle.dump(review_embedding, reviews_f)

In [10]:
# load embedding from saved
with open("../data/review_embedding_msmarco_v4.pkl", "rb") as reviews_f:
    review_embedding = pickle.load(reviews_f)

## Search reviews with sample query

In [19]:
# Sample query
query_embedding = model.encode('quiet and clean place close to beach in a safe neighborhood')

In [20]:
# find closest K matched reviews
k = 100

print('find top reviews that match query...')
raw_query_result = util.semantic_search(query_embedding, review_embedding, query_chunk_size=10000, top_k=k)
raw_query_result

find top reviews that match query...


[[{'corpus_id': 32194, 'score': 0.8698037266731262},
  {'corpus_id': 32611, 'score': 0.8518276810646057},
  {'corpus_id': 55459, 'score': 0.8205687999725342},
  {'corpus_id': 9604, 'score': 0.7918526530265808},
  {'corpus_id': 31781, 'score': 0.7841176390647888},
  {'corpus_id': 3002, 'score': 0.7814476490020752},
  {'corpus_id': 27033, 'score': 0.7802544832229614},
  {'corpus_id': 45068, 'score': 0.7781850695610046},
  {'corpus_id': 52666, 'score': 0.7752570509910583},
  {'corpus_id': 44716, 'score': 0.7714591026306152},
  {'corpus_id': 18823, 'score': 0.7683722376823425},
  {'corpus_id': 25000, 'score': 0.7642457485198975},
  {'corpus_id': 39757, 'score': 0.7555993795394897},
  {'corpus_id': 42621, 'score': 0.754262387752533},
  {'corpus_id': 56562, 'score': 0.7536784410476685},
  {'corpus_id': 26271, 'score': 0.7536617517471313},
  {'corpus_id': 38320, 'score': 0.7525529861450195},
  {'corpus_id': 25993, 'score': 0.7493453025817871},
  {'corpus_id': 42729, 'score': 0.748847782611846

In [21]:
# returns list of list, take only the first since there is only one query
query_result = pd.DataFrame(raw_query_result[0])
# select from reviews by result index
reviews_knn = reviews_selected.iloc[query_result['corpus_id']]
print(reviews_knn.shape)
reviews_knn.head(10)

(100, 79)


,id,date,reviewer_id,reviewer_name,comments,listing_url,scrape_id,last_scraped,source,name,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
listing_id,,,,,,,,,,,,,,,,,,,,,
49285727,649343418673735119,2022-06-14,966639,Jewett,Very clean space 👌 Conveniently close to the beach. Safe neighborhood.,https://www.airbnb.com/rooms/49285727,20220909140055,2022-09-09,city scrape,"5 MINUTES TO COAST! CLEAN,QUIET & SAFE",...,4.89,4.95,4.89,NaN,f,2,0,2,0,1.69
49564342,674729687726753024,2022-07-19,402554292,Efrain,Close to the beach nice and clean place.,https://www.airbnb.com/rooms/49564342,20220909140055,2022-09-10,city scrape,Family Friendly 2 BR Suite- 1 mi to the Beach,...,5.00,4.93,4.93,STR-2021-0034,f,1,0,1,0,2.93
658253971272445542,662424785848889067,2022-07-02,323946817,Kohath,A nice clean place in a quiet neighborhood,https://www.airbnb.com/rooms/658253971272445542,20220909140055,2022-09-09,city scrape,COZY ROOM WITH ONE BEDS,...,5.00,5.00,5.00,NaN,f,9,3,6,0,2.92
18026539,650026704497806015,2022-06-15,7966512,Magaly,"Very comfortable and clean! Great location for going to the beach, but also in a quiet neighborhood.",https://www.airbnb.com/rooms/18026539,20220909140055,2022-09-10,city scrape,A beautiful relaxing place near the ocean,...,4.96,4.96,4.96,STR-2021-0078,f,1,0,1,0,3.47
49002731,645672191096580315,2022-06-09,463119149,Simon,very clean and quiet neighborhood,https://www.airbnb.com/rooms/49002731,20220909140055,2022-09-09,city scrape,Private room w/queen bed for 1 -2 people.,...,5.00,5.00,5.00,NaN,f,6,1,5,0,0.17
4106488,699298612846874540,2022-08-22,17215922,Kelly,A lovely and clean place to stay in a quiet neighborhood!,https://www.airbnb.com/rooms/4106488,20220909140055,2022-09-09,city scrape,Gorgeous Master suite /Private bath / Pasadena,...,4.93,4.79,4.83,SRH20180089,f,3,1,2,0,0.86
44166594,664519533664723832,2022-07-05,117938121,Karen,"Nice clean, attractive place in a quiet neighborhood.",https://www.airbnb.com/rooms/44166594,20220909140055,2022-09-09,city scrape,"Home Away From Home, Los Angeles, Orange County",...,4.98,4.86,4.91,NaN,f,2,2,0,0,3.57
557170003593421901,666716641829406878,2022-07-08,181172249,Anthony,Clean and elegant. Great neighborhood and only a short drive to the beach,https://www.airbnb.com/rooms/557170003593421901,20220909140055,2022-09-10,city scrape,New Modern Private Entry 1 Bed/1Bath in West LA,...,4.98,4.95,4.95,HSR22-000073,t,1,0,1,0,8.73
632640524470310818,709466575881435294,2022-09-05,296694626,Carlos,"Very nice place, very close to the beach,Super clean I recommend 100%",https://www.airbnb.com/rooms/632640524470310818,20220909140055,2022-09-09,city scrape,Private Rm in Huntington Beach near Beaches&Disney,...,5.00,5.00,5.00,STR-2022-0100,f,1,0,1,0,2.24


In [22]:
# some top reviews may belong to the same listing
reviews_knn.index.value_counts()

19486721              1
26356811              1
52853695              1
43325886              1
49129149              1
                     ..
48870869              1
20591685              1
52672068              1
594078788008156284    1
543751971884337209    1
Name: listing_id, Length: 100, dtype: int64

In [23]:
# These are the top reviews that match user query
summary_knn = reviews_knn[['date','review_scores_rating','comments']]
summary_knn

,date,review_scores_rating,comments
listing_id,,,
49285727,2022-06-14,4.95,Very clean space 👌 Conveniently close to the beach. Safe neighborhood.
49564342,2022-07-19,5.00,Close to the beach nice and clean place.
658253971272445542,2022-07-02,5.00,A nice clean place in a quiet neighborhood
18026539,2022-06-15,4.99,"Very comfortable and clean! Great location for going to the beach, but also in a quiet neighborhood."
49002731,2022-06-09,5.00,very clean and quiet neighborhood
...,...,...,...
49343059,2022-08-13,5.00,Mark was really responsive and I found the stay to be overall great. The location is close to the beach and in a pretty safe neighborhood. The backyard is also quite nice.
17796465,2022-06-29,4.93,Nice and clean place to stay.
7998050,2022-07-22,4.92,"Nice place in a quiet neighborhood, but privacy was an issue."


In [24]:
# group reviews by listing_id (since there could be multiple reviews in KNN for the same listing)
summary_review_knn=reviews_knn.groupby(['listing_id']).size()
summary_review_knn=pd.DataFrame(summary_review_knn)

In [25]:
# merge with listings again to get selected top listings with features
top_listings = summary_review_knn.merge(listings_filtered, left_index=True, right_index=True)
top_listings.shape

(100, 75)

In [26]:
# selected listings based on query/review match
top_listings[['name','description','review_scores_rating']]

,name,description,review_scores_rating
listing_id,,,
1089331,Marina del Rey Beach Jr 1 Bdrm 0,"1 Bedroom Jr 3/4 Bath apartment located a block from the beach on the Marina del Rey Peninsula<br /><br /><b>The space</b><br />Very clean 1 Bedroom Jr. 3/4 Bath unit located a block from the beach in Marina del Rey, washer/dryer in the unit, a 3/4 kitchen (bar sink, refrigerator, 2 burner cooktop, and combination microwave/convection oven), wifi, 42"" smart tv.<br /><br />Walking distance to Venice Beach, lots of nice restaurants, Venice Canals. Santa Monica is 10 minutes driving. Good public transportation.<br /><br />This apartment does not have a parking space. Street parking only.<br /><br />Note: We only rent this unit from on or very close to the day that is becomes available so if you are interested, check our calendar and see if your dates are the same as the availability date.<br />Note: You have to rent a minimum of 30 days to comply with Los Angeles City Regulations. However, if you choose to rent 30 days, you will be assessed 14% tot tax. If you rent 31 days, you wil",4.89
2488270,Steps to Sand 29th St Back Cottage,"2 bdrm/1 bath (BACK) Beach Cottage (entrance off the back alley). Only four houses to the beach. <br /><br />Non-Smokers Only<br /><br />Minimum Rental Age - 25 yrs old (per City of NB)<br /><br />No pets, please no exceptions<br /><br />SMALL GARAGE- 1 space of shared 2 car garage is available for a compact car.<br />Maximum: (L) 190"", (H) 78"", (W) 73"" <br /><br />City of Newport Beach Short Term Lodging Permit# SLP13342<br /><br /><b>The space</b><br />Open concept full kitchen is completely outfit with all the essential cooking needs. Light and bright living room with comfortable seating, a 55' flat screen TV, local channels, Netflix and DVD player. Bathroom has a very large shower. Sink and separate vanity area comfortably accommodates two persons. Bath linens provided. Large bedroom closets, ceiling fans and window A/Cs in both bedrooms.<br /><br />Bedroom #1 - One King bed<br />Bedroom #2 - Two Twin XL beds<br /><br />This home has a maximum occupancy of 4 guests. Please do",4.96
3561523,Private Entrance Queen Bed Newport Mesa,"Hello,<br /> The room has its own private entrance.<br />The bathroom is shared with one other guest. <br />PLEASE DO NOT REQUEST WITHOUT 3 PRIOR POSITIVE REVIEWS. <br />The bio in your profile must be complete, tell about yourself.<br />Please see photos and read the entire listing before request to book. <br />Sorry, Non smokers ONLY that includes Pot<br />We offer a bedroom and the shared bathroom, there is no access to the rest of the home, kitchen or laundry<br /><br />Mon-Sat checkout is 9am, Sun 11am<br /> Thanks for the interest.<br /><br /><b>The space</b><br />Welcome to East Side Costa Mesa! <br />You will be staying in our Grandma cottage style home (built before 1945). <br />Your room has hard wood floors, and its own private entrance! The bathroom is shared with another guest when that room is occupied. It has a Very Tall Queen size bed, always clean sheets. There is plenty of closet space to store your stuff as well as a full length mirrors on the wall. There is also a",4.91
4106488,Gorgeous Master suite /Private bath / Pasadena,"This lovely home is centrally located in a desirable neighborhood of Pasadena. <br />Walking distance to market pharmacy and bank is less than 2 blocks <br /> <br />PLEASE READ THE ENTIRE<br />2 blocks to where The Rose Parades end <br /><br />Rose Bowl (4.2 miles),<br /><br />Caltech (1.7 miles), <br /><br />Huntington Library (2 miles), <br /><br />Old Pasadena ( 3.5 miles)<br /><br />JPL (7 miles). <br /><br />2.6 miles to Pasadena convention cente<br /><br /><b>The space</b><br />A master suite with private bathroom. <br />A queen size Opium bed that is one of a kind . <br /><br /><br />double french doors open to a deck and serene shaded backyard. <br />Cottage bungalow nestled amongst shaded trees and the majestic San Gabriel moun